In [19]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from bs4 import BeautifulSoup as bs
import requests
import unicodedata
from string import ascii_uppercase

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [20]:
def get_info(link):
    r = requests.get(link)
    soup = bs(r.content)
    info_tables = soup.findAll('table', attrs={'class': 'infobox vcard'})
    
    if len(info_tables) > 0:
        rows = info_tables[0].findAll('tr')

        info = {}
        for row in rows:
            k = row.find('th')
            if k and k.text != None:
                k = unicodedata.normalize('NFKD', k.text).strip().replace('\n', ' ')

            v = row.find('td')
            if v and v.text != None:
                v = unicodedata.normalize('NFKD', v.text).strip().replace('\n', ' ')
            else:
                v = 'N/A'


            if k != None:
                # print(k + ': ', v)
                info[k] = v
        return info

In [21]:
def get_company_links(company_links_page):
    r = requests.get(company_links_page)
    soup = bs(r.content)
    a_list = soup.findAll('table', attrs={'class': None})[0].findAll('a', href = lambda h: h and 'wiki' in h)
    return ['https://en.wikipedia.org' + a['href'] for a in a_list]

In [22]:
company_links_pages = ['https://en.wikipedia.org/wiki/Companies_listed_on_the_Toronto_Stock_Exchange_({})'.format(alphabet) for alphabet in ascii_uppercase]

links = []

for page in company_links_pages:
    links = links + get_company_links(page)

In [23]:
infos = [get_info(link) for link in links if get_info(link) != None]
infos

[{'Type': 'Private, with publicly traded income fund',
  'Traded as': 'TSX: AW.UN(A&W Revenue Royalties Income Fund)',
  'Industry': 'Fast food',
  'Founded': 'Winnipeg, Manitoba, Canada (1956; 64 years ago (1956))',
  'Headquarters': 'North Vancouver, British Columbia, Canada',
  'Number of locations': '850+[1]',
  'Key people': 'Susan Senecal (CEO & President)',
  'Products': 'Root beer, hamburgers, chicken burgers, veggie burgers, onion rings, french fries, sweet potato fries, breakfast items, and previously, hot dogs',
  'Revenue': '$99.4 million CAD (2016)[2]',
  'Net income': '$9.4 million CAD (2016)[2]',
  'Number of employees': '35,160 (2016)',
  'Parent': 'A&W Restaurants (1956–1972)Unilever (1972–1995)A&W Trade Marks Limited Partnership (Trademark only)[3]',
  'Website': 'aw.ca'},
 {'Type': 'Public',
  'Traded as': 'TSX: ABT',
  'Industry': 'Computer software',
  'Founded': '1993',
  'Headquarters': 'Vancouver, British Columbia, Canada',
  'Key people': 'Christy Wyatt (CEO), 

In [24]:
len(infos)

335

In [25]:
tsx = pd.DataFrame(infos)
tsx

,Type,Traded as,Industry,Founded,Headquarters,Number of locations,Key people,Products,Revenue,Net income,...,Details,Established,Location,Style,Owned by,No. of graves,Type of site,URL,Alexa rank,Defunct
0,"Private, with publicly traded income fund",TSX: AW.UN(A&W Revenue Royalties Income Fund),Fast food,"Winnipeg, Manitoba, Canada (1956; 64 years ago...","North Vancouver, British Columbia, Canada",850+[1],Susan Senecal (CEO & President),"Root beer, hamburgers, chicken burgers, veggie...",$99.4 million CAD (2016)[2],$9.4 million CAD (2016)[2],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Public,TSX: ABT,Computer software,1993,"Vancouver, British Columbia, Canada",NaN,"Christy Wyatt (CEO), Sean Maxwell (CCO)",Absolute,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Public,TSX: AAV,Oil and natural gas,"Calgary, Alberta (2001)","Calgary, Alberta",NaN,"Andy J. Mah, CEO, Mike Belenkie, President & C...",Oil and natural gas,225 million CAD (2018),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Public,TSX: ARE,"Construction, Infrastructure Development,","1877 Hamilton, Ontario","Toronto, Calgary, Vancouver, Canada",NaN,Jean-Louis Servranckx President and CEO John ...,Asphalt,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Public,TSX: AEMNYSE: AEMS&P/TSX 60 component,Mining & Gold Producers & Products Manufacturing,1953 as Cobalt Consolidated Mining Company,"Toronto, Ontario, Canada",NaN,James D. Nasso(Chairman of the Board)Sean Boyd...,NaN,$ 2.875 billion (2016)[1],$ 214 million (2016)[1],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
330,Public,TSX: WPK,Packaging & Containers,NaN,"Winnipeg, Manitoba, Canada",10,James Holland,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
331,Public,TSX: WSP S&P/TSX Composite Component,"Design, Engineering consulting, Environmental ...",1959; 61 years ago (1959),"Montreal, Quebec, Canada",NaN,Alexandre L'Heureux(President and CEO),NaN,C$6.4 billion (2016)[1],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
332,Public,TSX: YRINYSE: AUY,Mining,"March 17, 1994","Toronto, Ontario, Canada",NaN,"Victor Bradley (CEO, 1994-2003)Peter Marrone (...","Gold, silver",$1.612 billion (2019) $1.799 billion (2018) $1...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
333,Public,TSX: Y,Media,1908; 112 years ago (1908) (as Yellow Media),"1751 Richardson St., Montreal, Quebec, Canada",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
